In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import image_dataset_from_directory,to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.image import resize
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split
import numpy as np
def load_dataset(dir,image_size=(32,32)):
    data=image_dataset_from_directory(dir,image_size=image_size)
    images,labels=[],[]
    for x,y in data:
        images.append(x.numpy()/255.0)
        labels.append(y.numpy())
    X=np.concatenate(images)
    Y=np.concatenate(labels)
    Y=to_categorical(Y,10)
    return X,Y

def preprocess(X,batch=200):
    finished=[]
    for i in range(0,len(X),batch):
        packet=X[i:i+batch]
        sized=resize(packet,(224,224))
        processed=preprocess_input(sized.numpy())
        finished.append(processed)
    return (np.concatenate(finished))
X,Y=load_dataset("D:\\Source_code\\Lab\\data\\cifar10\\test")

X=preprocess(X)

xtrain,xtest,ytrain,ytest=train_test_split(X,Y,random_state=42,test_size=0.3)
base_model=VGG19(include_top=False)
base_model.trainable=False
model=Sequential([
    base_model,
    Flatten()
    Dense(224),
    Dense(10,activation='softmax')
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train
history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=5, batch_size=16)

# Plot
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Val")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
